In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
pd.options.mode.chained_assignment = None  # default='warn'
from scipy.spatial import distance as dist
import scipy.cluster.hierarchy as hier
import os
import numpy as np

file_path = './test_files/3Dblood27.11.txt'
df = pd.read_csv(file_path, header=None)
df.columns = ['x', 'y', 'z', 'r', 'g', 'b']
df

,x,y,z,r,g,b
0,4.300000,11.197390,3.047941,0,0,0
1,4.200000,11.197390,3.047941,0,0,0
2,4.100000,11.197390,3.047941,0,0,0
3,4.000000,11.197390,3.047941,0,0,0
4,3.900000,11.197390,3.047941,0,0,0
...,...,...,...,...,...,...
1799,4.324351,10.356519,3.886832,255,0,255
1800,4.253031,10.706822,3.538061,255,0,255
1801,4.260648,10.636221,3.608462,255,0,255
1802,4.295052,11.128907,3.116422,255,0,255


In [6]:
def distance_calculator(df):
    dx = df['x'].diff()
    dy = df['y'].diff()
    dz = df['z'].diff()

    # Calculate the Euclidean distance between consecutive rows
    distances = np.sqrt(dx**2 + dy**2 + dz**2)
    df['distance_from_last'] = distances
    return df

def remove_overlap(shape):
    # Checks for any large jumps at the end of a line and, if found, moves the last row to the top
    # This hopefully removes the jump and makes the line continuous
    # Note this seems very fragile and I probably need add a second check for large jumps at the end of the code
    if shape.iloc[-1]['distance_from_last'] > 2.:
        # Move the last row to the top
        last_row = shape.iloc[[-1]]  # Select the last row as a DataFrame
        remaining_rows = shape.iloc[:-1]  # Select all rows except the last
        shape = pd.concat([last_row, remaining_rows]).reset_index(drop=True)

    shape = distance_calculator(shape)
    return shape

def midlinejumpsplitter(shape):
    # This needs generalising to lines with more than 2 jumps
    id = shape['line_id'].unique()[0]
    # Split the line at the index - at the moment uses a completely arbitrary distance of 2mm
    split_index = shape[shape['distance_from_last'] > 2.].index[0]
    shape1 = shape.iloc[:split_index]
    shape1['line_id'] = 1
    shape1['distance_from_last'].iloc[0] = np.nan
    shape2 = shape.iloc[split_index:]
    shape2['line_id'] = 2
    shape2['distance_from_last'].iloc[0] = np.nan
    return shape1, shape2

def shapesplitter(df):
    # Identify line IDs that have large jumps in the middle
    line_ids = np.sort(df['line_id'].unique())   # Sorting makes life easier later
    line_ids_new = line_ids.copy()   # A list of line IDs that we're going to update
    for line_id in line_ids:
        shape = df[df['line_id'] == line_id]
        if shape['distance_from_last'].max() > 2.:
            shape1, shape2 = midlinejumpsplitter(shape)
            df  = df[df['line_id'] != line_id] 
            line_ids_new = line_ids_new[line_ids_new != line_id]
            line_ids_new = np.append(line_ids_new, [line_ids_new[-1]+1, line_ids_new[-1]+2])
            shape1['line_id'], shape2['line_id'] = line_ids_new[-2], line_ids_new[-1]
            shape = pd.concat([shape1, shape2])
            df = pd.concat([df, shape])  
    return df

def node_finder(df):
    # Use hierarchical clustering to note common start / end points
    # Calculate node positions based on cluster centroids
    # Append centroids to start / end of each line
    start_points = df.groupby('line_id').first() # First point of each line
    end_points = df.groupby('line_id').last() # Last
    terminal_points = pd.concat([start_points, end_points])   # Combine the two
    dist_mat = dist.pdist(terminal_points[['x', 'y', 'z']].values)   
    link_mat = hier.linkage(dist_mat)
    # fcluster assigns each of the particles in positions a cluster to which it belongs
    cluster_idx = hier.fcluster(link_mat, t=1, criterion='distance')   # t defines the max cophonetic distance in a cluster
    terminal_points['cluster'] = cluster_idx

    # Calculate the mean position of each cluster
    nodes = terminal_points.groupby('cluster').mean()
    for n in terminal_points.index.unique():
        clusters = terminal_points.loc[n]['cluster']
        for c in clusters:
            new_point = nodes.loc[c:c]
            line = df[df['line_id'] == n]
            line_start = line.head(1)
            line_end = line.tail(1)
            new_point[['r', 'g', 'b', 'line_id']] = line_start[['r', 'g', 'b', 'line_id']].values

            start_sep = dist.euclidean(new_point[['x', 'y', 'z']].values[0], line_start[['x', 'y', 'z']].values[0])
            end_sep = dist.euclidean(new_point[['x', 'y', 'z']].values[0], line_end[['x', 'y', 'z']].values[0])
            if start_sep < end_sep:
                line = pd.concat([new_point, line]) 
            elif start_sep > end_sep:
                line = pd.concat([line, new_point])
            
            df = df[df['line_id'] != n]
            df = pd.concat([df, line])
            df = df.reset_index(drop=True)
            df = distance_calculator(df)
            
            # Set distance from last to NaN for the first row of each line
            df.loc[df.groupby('line_id').head(1).index, 'distance_from_last'] = np.nan
    return df, terminal_points, nodes

def line_plotter(df):
    # Plots lines assigning a colour to each line_id
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    for n in np.unique(df['line_id'].values):
        ax.plot(df[df['line_id'] == n]['x'], df[df['line_id'] == n]['y'], df[df['line_id'] == n]['z'])

    # If you need a specific line plotting
    # n = 2
    # ax.plot(df[df['line_id'] == n]['x'], df[df['line_id'] == n]['y'], df[df['line_id'] == n]['z']

    for n in np.unique(terminal_points['cluster'].values):
        ax.scatter(terminal_points[terminal_points['cluster'] == n]['x'], terminal_points[terminal_points['cluster'] == n]['y'], terminal_points[terminal_points['cluster'] == n]['z'])

    ax.view_init(elev=30, azim=60)  # Elevation of 30 degrees, azimuth of 45 degrees
    plt.show()


def eulerficator(df, terminal_points, nodes):
    terminal_points_nogroups = terminal_points.reset_index()
    clusters = terminal_points_nogroups['cluster'].unique()
    lines = terminal_points_nogroups['line_id'].unique()

    # Run through each cluster and create two dictionaries 
    # 1 - cluster numbers as key and the connecting lines as values
    # 2 - cluster numbers as key and the number of connecting nodes as values
    cluster_dict = {}
    connectivity_dict = {}
    for c in clusters:
        connecting_lines = terminal_points_nogroups[terminal_points_nogroups['cluster'] == c]
        cluster_dict[c] = list(connecting_lines['line_id'].values)

        connectivity_dict[c] = len(connecting_lines)

    line_order = []
    line_order_grouped = []
    node_order = [] 
    node_order_grouped = []
    while len(line_order) < len(lines):
        current_line_lines = []
        current_line_nodes = []

        # Sort lines by height order - bottom up
        min_z = df.groupby('line_id')['z'].min()
        heightsorted_line_ids = min_z.sort_values().index.tolist()
        unprinted_lines = [n for n in heightsorted_line_ids if n not in line_order]

        # Pick the unprinted line with the lowest z-value to start with
        next_line = unprinted_lines[0]
        line_order.append(next_line)
        current_line_lines.append(next_line)
        unprinted_lines = unprinted_lines[1:] # Remove the printed line from the list of remaining lines

        # Calculate which node you're at - it's the other node to which next_line is connected
        connected_nodes = terminal_points.loc[next_line]['cluster'].values

        # Start at node with lower z-value - record as first node of this line
        start_node = nodes.loc[connected_nodes]['z'].idxmin()
        current_line_nodes.append(start_node)
        node_order.append(start_node)

        # Find node at other end of line
        end_node = connected_nodes[connected_nodes != start_node][0]
        current_line_nodes.append(end_node)
        node_order.append(end_node)

        connected_lines = cluster_dict[end_node]   # Lines connected to end node
        connected_lines = [n for n in unprinted_lines if n in connected_lines]   # List unprinted connected lines in height-order
        while len(connected_lines) > 0:
            # Pick next line based on lowest z_min
            next_line = connected_lines[0]
            line_order.append(next_line)
            current_line_lines.append(next_line)
            unprinted_lines.remove(next_line)

            # Calculate which node you've moved to
            connected_nodes = terminal_points.loc[next_line]['cluster'].values
            start_node = end_node
            end_node = connected_nodes[connected_nodes != start_node][0]
            current_line_nodes.append(end_node)
            node_order.append(end_node)

            connected_lines = cluster_dict[end_node]

            # Remove lines that have already been printed
            connected_lines = [n for n in unprinted_lines if n in connected_lines]
        
        line_order_grouped.append(current_line_lines)
        node_order_grouped.append(current_line_nodes)

    return line_order, node_order, line_order_grouped, node_order_grouped


def line_order_corrector(df, line_order, line_order_grouped, nodes, node_order_grouped):
    # Make lines run in node order.
    df = df.set_index('line_id').loc[line_order].reset_index()   # Reorder the dataframe based on the line order

    for idx_path, path in enumerate(line_order_grouped):
        for idx_line, line in enumerate(path):
            start_node = node_order_grouped[idx_path][idx_line]
            line_start = df[df['line_id'] == line].iloc[0][['x', 'y', 'z']].values 
            node_loc = nodes.loc[start_node][['x', 'y', 'z']].values

            if not np.array_equal(line_start, node_loc):
                print('Reversing line ', line)
                new_line = df[df['line_id'] == line].iloc[::-1]
                df = df[df['line_id'] != line]
                df = pd.concat([df, new_line])

    df = df.set_index('line_id').loc[line_order].reset_index()   # Reorder the dataframe based on the line order
    df = distance_calculator(df)
    return df

# Calculate the distance between consecutive points
df = distance_calculator(df)
# If line ID column doesn't exist, assign line IDs based on RGB values
if 'line_id' not in df.columns:
    df['line_id'] = pd.factorize(df[['r','g','b']].apply(tuple, axis=1))[0]
# Remove large jumps at the end of lines
df = df.groupby('line_id', group_keys=False).apply(remove_overlap)

# Recalculate the distance between consecutive points
df = distance_calculator(df)

# Set distance from last to NaN for the first row of each line
df.loc[df.groupby('line_id').head(1).index, 'distance_from_last'] = np.nan

# Find and split lines that have big jumps in the middle, e.g., inlet and outlet lines
df = shapesplitter(df)

# Set distance from last to NaN for the first row of each line
df.loc[df.groupby('line_id').head(1).index, 'distance_from_last'] = np.nan

# Find nodes using hierarchical clustering
df, terminal_points, nodes = node_finder(df)

# Calculate line order via z-weighted Fleury-ish algorithm
line_order, node_order, line_order_grouped, node_order_grouped = eulerficator(df, terminal_points, nodes)
print(line_order)

# Correct line order to run in node order
df = line_order_corrector(df, line_order, line_order_grouped, nodes, node_order_grouped)

[2, 3, 1, 6, 10, 4, 5, 7, 8, 9]
Reversing line  3
Reversing line  6
Reversing line  5
Reversing line  8
Reversing line  9


In [7]:
def e_calculator(df, d=1):
    alpha = 0.7034   # Extrusion multiplier
    df = df.fillna(0)
    df['V_mL'] = np.pi*((d/2)**2)*df['distance_from_last']
    df['E'] = np.pi*(1/alpha)*df['V_mL']  # Amount to extrude

    # Set the extrusion amount for the first line of each path to zero
    for path in line_order_grouped:
        index_to_update = df[df['line_id'] == path[0]].index[0]  # Find the index of the first match
        df.loc[index_to_update, 'E'] = 0  # Update the value at that index

    df['E_cumulutative'] = df['E'].cumsum()
    return df

def first_line(df):
    first_line = positioning(df[df['line_id'] == path[0]].iloc[0], f_print)
    with open("gcode.txt", "a") as file:
        file.write(first_line)

def preamble():
    preamble_out = """; Setup section
M82 ; absolute extrusion mode
G90 ; use absolute positioning
M104 S0.0 ; Set Hotend Temperature to zero
M140 S0.0 ; set bed temp
M190 S0.0 ; wait for bed temp
G28 ; home all
G92 E0.0 ; Set zero extrusion
M107 ; Fan off

G1 X97.5 Y147 F2000 ; Move printhead to centre of printbed
G92 X0 Y0 E0 ; Set zero extrusion"""
    with open("gcode.txt", "w") as file:
        file.write(preamble_out)

def postamble():
    postamble_out = """\n
; End of print
M140 S0 ; Set Bed Temperature to zero
M107 ; Fan off
M140 S0 ; turn off heatbed
M107 ; turn off fan
G1 X178 Y180 F4200 ; park print head
G28 ; Home all
M84 ; disable motors
M82 ; absolute extrusion mode
M104 S0 ; Set Hotend Temperature to zero
; End of Gcode"""
    with open("gcode.txt", "a") as file:
        file.write(postamble_out)

def position_printhead(df, line_id, f_print):
    first_line = df[df['line_id'] == line_id].iloc[0]
    positioning = """\n
; Initial positioning for new print path
G1 F800          ; Printhead speed for initial positioning
G1 X{} Y{}       ; XY-coords of first point of path
G1 Z{}           ; Z-coord of first point of path
G4 S2            ; Dwell for 2 seconds for karma / aligment
G1 F{}           ; Set printhead speed""".format(first_line['x'], first_line['y'], first_line['z'], f_print)
    with open("gcode.txt", "a") as file:
        file.write(positioning)

def print_line(df, line_id):
    line = df[df['line_id'] == line_id]
    start_line = "\n\n; Start of line number: " + str(line_id) + "\n"
    gcode_output = "\n".join(
        "G1 X" + line['x'].astype(str) + " Y" + line['y'].astype(str) + " Z" + line['z'].astype(str) + " E" + line['E'].astype(str))
    with open("gcode.txt", "a") as file:
        file.write(start_line)
        file.write(gcode_output)

def raise_printhead(df):
    # Give 5 mm clearance
    raise_printhead_out = """\n
; Raise printhead
G1 Z{} F1000""".format(5+df['z'].max())
    with open("gcode.txt", "a") as file:
        file.write(raise_printhead_out)

# Settings
f_print = 500    # Printhead speed in mm / min
d = 0.1   # Target fibril diameter in mm

# Start of print code
df = df[df['distance_from_last'] != 0.]

# Calculate extrusion amount between points
df_print = e_calculator(df, 0.1)
df_print = df_print.round(3)

# Write sections to file
preamble()
for path in line_order_grouped:
    position_printhead(df_print, path[0], f_print)
    for line_id in path:
        print_line(df_print, line_id)
    raise_printhead(df_print)
postamble()

# And modify E_calculator for Sara's calibration

    

In [9]:
floor = -61

df

,line_id,x,y,z,r,g,b,distance_from_last
0,2,18.111596,11.182686,-58.013391,0,127,0,NaN
1,2,18.110164,11.386743,-58.186220,0,127,0,0.267416
2,2,18.138710,11.315074,-58.122592,0,127,0,0.099999
3,2,18.051130,11.532608,-58.309650,0,127,0,0.299970
4,2,18.080977,11.459302,-58.248533,0,127,0,0.099999
...,...,...,...,...,...,...,...,...
1819,9,4.000000,11.197390,-57.952059,0,0,0,0.100000
1820,9,4.100000,11.197390,-57.952059,0,0,0,0.100000
1821,9,4.200000,11.197390,-57.952059,0,0,0,0.100000
1822,9,4.300000,11.197390,-57.952059,0,0,0,0.100000
